In [ ]:
import os
import matplotlib.pyplot as plt
from pathlib import Path
from diopter.compiler import SourceProgram, Language, CompilerExe, CompilationSetting, OptLevel
from utils import get_ratio, get_binary_size
import seaborn as sns

sns.set_theme(style="ticks")

In [ ]:
COMPILER = {
    "gcc": CompilerExe.get_system_gcc(),
    "clang": CompilerExe.get_system_clang(),
}

def get_compilation_setting(compiler, opt_level):
    return CompilationSetting(
        compiler=COMPILER[compiler],
        opt_level=OptLevel.from_str(opt_level),
        flags=("-march=native",),
    )

def load_settings(settings_path):
    experiment_settings = {}
    with open(settings_path, "r") as f:
        for line in f:
            line = line.strip()
            if line.startswith("Experiment settings:"):
                continue
            if line:
                key, value = line.split(": ", 1)
                experiment_settings[key] = value


    return experiment_settings

def get_ratios(experiment_dir):
    experiment_settings_path = os.path.join(experiment_dir, "settings.log")
    experiment_settings = load_settings(experiment_settings_path)
    compilation_setting = CompilationSetting(
        compiler=COMPILER[experiment_settings["compiler"]],
        opt_level=OptLevel.from_str(experiment_settings["opt_level"]),
        flags=("-march=native",),
    )

    ratios = []

    for iteration_dir in os.listdir(experiment_dir):
        iteration_path = Path(experiment_dir) / iteration_dir
        if iteration_path.is_dir():
            step = int(iteration_dir.split("_")[1])
            c_program_path = iteration_path / "best.c"
            if c_program_path.exists():
                with open(c_program_path, "r") as f:
                    code = f.read()
                p = SourceProgram(
                    code=code,
                    language=Language.C,
                )
                ratios.append((step,get_ratio(p, compilation_setting)))
    return sorted(ratios)

In [ ]:
main_program = SourceProgram(code="int main() {}", language=Language.C)
main_ratio_gcc_O3 = get_ratio(main_program, get_compilation_setting("gcc", "O3"))
main_ratio_gcc_Os = get_ratio(main_program, get_compilation_setting("gcc", "Os"))
main_ratio_clang_O3 = get_ratio(main_program, get_compilation_setting("clang", "O3"))
main_ratio_clang_Os = get_ratio(main_program, get_compilation_setting("clang", "Os"))

In [ ]:
sns.set_theme(style="ticks")

experiment_root = "/home/remo/Documents/eth/fs23/ast2023/out"

# Create subplots
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(8, 7))

# Iterate over experiment directories
for experiment_dir in os.listdir(experiment_root):
    name = experiment_dir
    experiment_dir = os.path.join(experiment_root, experiment_dir)
    settings = load_settings(os.path.join(experiment_dir, "settings.log"))
    #name = f'{settings["compiler"]} -{settings["opt_level"]}, threshold: {settings["threshold"]}s, timeout: {settings["timeout"]}s'
    ratios = get_ratios(experiment_dir)
    x, y = zip(*ratios)
    
    if settings["compiler"] == "clang":
        if settings["opt_level"] == "O3":
            ax1.plot(x, y, label=name)
        else:
            ax2.plot(x, y, label=name)
    elif settings["compiler"] == "gcc":
        if settings["opt_level"] == "O3":
            ax3.plot(x, y, label=name)
            print(ratios)
        else:
            ax4.plot(x, y, label=name)

# Set labels and legend for the first subplot
ax1.set_xlabel("Round")
ax1.set_ylabel("Ratio")
ax1.set_title("Clang -O3")
ax1.axhline(main_ratio_clang_O3, color="red", linestyle="--")
ax1.legend()

# Set labels and legend for the second subplot
ax2.set_xlabel("Round")
ax2.set_ylabel("Ratio")
ax2.set_title("Clang -Os")
ax2.axhline(main_ratio_clang_Os, color="red", linestyle="--")
ax2.legend()

# Set labels and legend for the first subplot
ax3.set_xlabel("Round")
ax3.set_ylabel("Ratio")
ax3.set_title("GCC -O3")
ax3.axhline(main_ratio_gcc_O3, color="red", linestyle="--")
ax3.legend()

# Set labels and legend for the second subplot
ax4.set_xlabel("Round")
ax4.set_ylabel("Ratio")
ax4.set_title("GCC -Os")
ax4.axhline(main_ratio_gcc_Os, color="red", linestyle="--")
ax4.legend()

# Adjust layout and display the plot
plt.tight_layout()
sns.despine()
plt.show()